In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
import csv
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avime\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avime\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avime\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [58]:
import io
data_string = """Reviews, Rating
                 this is my sad first review, 1
                 how bad can things get, 2
                 I feel super happy about this review and good about me, 3"""

data = io.StringIO(data_string)
df = pd.read_csv(data, sep=",")
df

,Reviews,Rating
0,this is my sad first review,1
1,how bad can things get,2
2,I feel super happy about this review and good about me,3


In [59]:
def normalizer(review):
    only_letters = re.sub("[^a-zA-Z]", " ",review) 
    tokens = nltk.word_tokenize(only_letters)[2:]   # Tokenize data by words
    lower_case = [l.lower() for l in tokens]    # Converting data into lowercase
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))   # Removing stopwords
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]   # Lemmatizing the data  
    return lemmas

In [60]:
normalizer("I don't like this. I feel like I am depressed.")

['like', 'feel', 'like', 'depressed']

In [61]:
pd.set_option('display.max_colwidth', -1) # Setting this so we can see the full content of cells
df['new'] = df.Reviews.apply(normalizer)
df[['Reviews','new']].head()

<ipython-input-61-8b5e88e2488b>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) # Setting this so we can see the full content of cells


,Reviews,new
0,this is my sad first review,"[sad, first, review]"
1,how bad can things get,"[thing, get]"
2,I feel super happy about this review and good about me,"[super, happy, review, good]"


In [62]:
from nltk import ngrams
def ngrams(input_list):
    onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams
    # return bigrams+trigrams
    # return onegrams
df['grams'] = df.new.apply(ngrams)
df[['grams']].head()

,grams
0,"[sad first, first review]"
1,[thing get]
2,"[super happy, happy review, review good]"


In [66]:
#attach loop here to fetch all the values and store in an array corresponding to the review

for i in (0,2):
    analysisPol=TextBlob(df.Reviews[i]).polarity
    analysisSub=TextBlob(df.Reviews[i]).subjectivity
    print(df.Reviews[i])
    print('\n')
    print(analysisPol)
    print(analysisSub)

                 this is my sad first review


-0.125
0.6666666666666666
                 I feel super happy about this review and good about me


0.611111111111111
0.7555555555555555


In [7]:
import requests
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from datetime import datetime
from time import sleep
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import math

In [3]:
# code adjustments

rat_links=[]
with open('ratings_links.txt') as csvfile:
    csvReader = csv.reader(csvfile)
    for row in csvReader:
        if row ==[]:
            break
        rat_links.append(row[0])
rat_links



['https://www.amazon.in/Test-Exclusive_2020_1180-Multi-3GB-Storage/product-reviews/B089MT34QL/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1',
 'https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1']

In [4]:
df = pd.DataFrame()

#df['Product Name']=[]
df['Names']=[]
df['Heading']=[]
df['Date']=[]
df['Ratings']=[]
df['Reviews']=[]
df['Page Number']=[]
df['Link']=[]

In [5]:
for i in range(1,len(rat_links)):
    url = rat_links[i]
    l = len(url)
    url= url[:l-1]
    pg=[]
    link=[]
    n=[]
    rh=[]
    r=[]
    ra=[]
    dat=[]
    
    for j in range(1,60,4):
        url_n = url+str(j)
        print(url_n)
        page = requests.get(url_n)
        soup = BeautifulSoup(page.content,'html.parser')
        
        #names
        names = soup.find_all('span',class_='a-profile-name')
        cust_name = []
        for i in range(0,len(names)):
            cust_name.append(names[i].get_text())
        cust_name.pop(0)
        cust_name.pop(0)
        n.append(cust_name)
        
        #date of review
        date = soup.find_all('span',class_='review-date')
        rdate=[]
        for i in date:
            rdate.append(i.get_text())
        rdate.pop(0)
        rdate.pop(0)
        dat.append(rdate)
        
        #review title
        review_head = soup.find_all("a",class_='review-title')
        review=[]
        for i in review_head:
            review.append(i.get_text())
        review_t = [titles.lstrip('\n') for titles in review]
        review_t = [titles.rstrip("\n") for titles in review_t]
        rh.append(review_t)
        
        #page and links
        pages=[j]*(len(review_t))
        lnk = [url]*(len(review_t))
        #itm= df1["Product"][0]
        pg.append(pages)
        link.append(lnk)
        
        #review Body
        reviews=soup.find_all('span',{'data-hook':'review-body'})
        review_content = []
        for i in range(0,len(reviews)):
            review_content.append(reviews[i].get_text())
        review_content[:] = [reviews.lstrip('\n') for reviews in review_content]
        review_content[:] = [reviews.rstrip('\n') for reviews in review_content]
        r.append(review_content)
        
        #review Rating
        rating = soup.find_all('i',class_='review-rating')
        rate = []
        for i in range(0,len(rating)):
            rate.append(rating[i].get_text())
        rate.pop(0)
        rate.pop(0)
        ra.append(rate)
    
    names = np.concatenate(n)
    review_t = np.concatenate(rh)
    review_c = np.concatenate(r)
    ratt= np.concatenate(ra)
    pag = np.concatenate(pg)
    llin= np.concatenate(link)
    date_r=np.concatenate(dat)
 
    #prodd = [itm]*len(review_c)

https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=13
https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=17
https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08L5TNJHG/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=21
https://www.amazon.in/New-Apple-iPhone-12-128GB/product-reviews/B08

In [13]:
# CLEANING TEST

In [40]:
df = pd.concat(map(pd.read_csv, ['Data_1', 'Data_1b','Data_1c']))
df

,Product Name,Names,Heading,Date,Ratings,Reviews,Page Number,Link
0,OnePlus 9R 5G (Lake Blue 8GB RAM 128GB Storage),ASHANNITA,Claim for Replacement of Oneplus 9R phone...,Reviewed in India on 30 May 2021,1.0 out of 5 stars,HiPlease consider this message with a seriou...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
1,OnePlus 9R 5G (Lake Blue 8GB RAM 128GB Storage),Vijay Gandhi,OnePlus 9R - Not As per Expectations,Reviewed in India on 19 April 2021,1.0 out of 5 stars,Battery Issue.... Phone is Heating during Ch...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
2,OnePlus 9R 5G (Lake Blue 8GB RAM 128GB Storage),NeHaaL,Heating issue,Reviewed in India on 19 April 2021,1.0 out of 5 stars,Facing heating issue while using camera app ...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
3,OnePlus 9R 5G (Lake Blue 8GB RAM 128GB Storage),Mohit MJ,Amazing - True Flagship,Reviewed in India on 19 April 2021,5.0 out of 5 stars,My first time buying a OnePlus phone and I r...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
4,OnePlus 9R 5G (Lake Blue 8GB RAM 128GB Storage),AshokA,Over heat mobile,Reviewed in India on 19 April 2021,1.0 out of 5 stars,Within 10 minutes of usage. It’s felt like o...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
...,...,...,...,...,...,...,...,...
145,Redmi Note 9 (Aqua Green 4GB RAM 128GB Storage...,Hansa Motwani,Poor Quality Phone - NOT RECOMMENDED,Reviewed in India on 22 September 2020,2.0 out of 5 stars,Screen quality and build is pretty ordinary....,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
146,Redmi Note 9 (Aqua Green 4GB RAM 128GB Storage...,Muslim ponnu's view,Battery & picture quality,Reviewed in India on 1 October 2020,5.0 out of 5 stars,The processor is a Qualcomm Snapdragon 720G ...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
147,Redmi Note 9 (Aqua Green 4GB RAM 128GB Storage...,Karan,Don't focus on the bad and critical reviews....,Reviewed in India on 15 October 2020,5.0 out of 5 stars,i loved the phone basically at the price po...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
148,Redmi Note 9 (Aqua Green 4GB RAM 128GB Storage...,Amazon Customer,Items missing in product very disappointed,Reviewed in India on 22 June 2021,1.0 out of 5 stars,Product is from Seller - Dakshita Etel. I am...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...


In [41]:
ext=[]
for i in range(len(df['Product Name'])):
    name , extra = df.iloc[i,0].split("(")
    df.iloc[i,0] = name
    ext.append(extra)


In [43]:
df.tail()

,Product Name,Names,Heading,Date,Ratings,Reviews,Page Number,Link
145,Redmi Note 9,Hansa Motwani,Poor Quality Phone - NOT RECOMMENDED,Reviewed in India on 22 September 2020,2.0 out of 5 stars,Screen quality and build is pretty ordinary....,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
146,Redmi Note 9,Muslim ponnu's view,Battery & picture quality,Reviewed in India on 1 October 2020,5.0 out of 5 stars,The processor is a Qualcomm Snapdragon 720G ...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
147,Redmi Note 9,Karan,Don't focus on the bad and critical reviews....,Reviewed in India on 15 October 2020,5.0 out of 5 stars,i loved the phone basically at the price po...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
148,Redmi Note 9,Amazon Customer,Items missing in product very disappointed,Reviewed in India on 22 June 2021,1.0 out of 5 stars,Product is from Seller - Dakshita Etel. I am...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
149,Redmi Note 9,VIMAL CHAARAN,Redmi note 9,Reviewed in India on 22 September 2020,5.0 out of 5 stars,JUST WRITING AFTER 1 WEEK OF USAGE WITH JIO ...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...


In [44]:
# Dates clean

for i in range(len(df["Date"])):
    rw= df.iloc[i,3]
    temp = [ x for x in rw.split(" ")]
    tem  = str(temp[4:])
    st=" ".join(temp[4:])
    #print(st)
    df.iloc[i,3] = st

# Cleaning Ratings

for i in range(len(df["Ratings"])):
    rat = df.iloc[i, 4]
    df.iloc[i,4]= float(rat[0:3])
df  

,Product Name,Names,Heading,Date,Ratings,Reviews,Page Number,Link
0,OnePlus 9R 5G,ASHANNITA,Claim for Replacement of Oneplus 9R phone...,30 May 2021,1,HiPlease consider this message with a seriou...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
1,OnePlus 9R 5G,Vijay Gandhi,OnePlus 9R - Not As per Expectations,19 April 2021,1,Battery Issue.... Phone is Heating during Ch...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
2,OnePlus 9R 5G,NeHaaL,Heating issue,19 April 2021,1,Facing heating issue while using camera app ...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
3,OnePlus 9R 5G,Mohit MJ,Amazing - True Flagship,19 April 2021,5,My first time buying a OnePlus phone and I r...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
4,OnePlus 9R 5G,AshokA,Over heat mobile,19 April 2021,1,Within 10 minutes of usage. It’s felt like o...,1,https://www.amazon.in/Test-Exclusive_2020_1180...
...,...,...,...,...,...,...,...,...
145,Redmi Note 9,Hansa Motwani,Poor Quality Phone - NOT RECOMMENDED,22 September 2020,2,Screen quality and build is pretty ordinary....,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
146,Redmi Note 9,Muslim ponnu's view,Battery & picture quality,1 October 2020,5,The processor is a Qualcomm Snapdragon 720G ...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
147,Redmi Note 9,Karan,Don't focus on the bad and critical reviews....,15 October 2020,5,i loved the phone basically at the price po...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
148,Redmi Note 9,Amazon Customer,Items missing in product very disappointed,22 June 2021,1,Product is from Seller - Dakshita Etel. I am...,15,https://www.amazon.in/Redmi-Note-Green-128GB-S...
